In [1]:
print("hi")

hi


# Unsloth

In [2]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

# 4bit pre quantized models we support for 4x faster downloading + no OOMs.
fourbit_models = [
    "unsloth/Meta-Llama-3.1-8B-bnb-4bit",      # Llama-3.1 15 trillion tokens model 2x faster!
    "unsloth/Meta-Llama-3.1-8B-Instruct-bnb-4bit",
    "unsloth/Meta-Llama-3.1-70B-bnb-4bit",
    "unsloth/Meta-Llama-3.1-405B-bnb-4bit",    # We also uploaded 4bit for 405b!
    "unsloth/Mistral-Nemo-Base-2407-bnb-4bit", # New Mistral 12b 2x faster!
    "unsloth/Mistral-Nemo-Instruct-2407-bnb-4bit",
    "unsloth/mistral-7b-v0.3-bnb-4bit",        # Mistral v3 2x faster!
    "unsloth/mistral-7b-instruct-v0.3-bnb-4bit",
    "unsloth/Phi-3-mini-4k-instruct",          # Phi-3 2x faster!
    "unsloth/Phi-3-medium-4k-instruct",
    "unsloth/gemma-2-9b-bnb-4bit",
    "unsloth/gemma-2-27b-bnb-4bit",            # Gemma 2x faster!
    "unsloth/gemma-2-2b-bnb-4bit",             # New small Gemma model!
] # More models at https://huggingface.co/unsloth

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/gemma-2-9b",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
==((====))==  Unsloth 2024.8: Fast Gemma2 patching. Transformers = 4.44.2.
   \\   /|    GPU: NVIDIA GeForce RTX 2060. Max memory: 6.0 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.4.0+cu121. CUDA = 7.5. CUDA Toolkit = 12.1.
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.27.post2. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth


model.safetensors:   0%|          | 0.00/6.13G [00:00<?, ?B/s]

ValueError: Some modules are dispatched on the CPU or the disk. Make sure you have enough GPU RAM to fit the quantized model. If you want to dispatch the model on the CPU or the disk while keeping these modules in 32-bit, you need to set `load_in_8bit_fp32_cpu_offload=True` and pass a custom `device_map` to `from_pretrained`. Check https://huggingface.co/docs/transformers/main/en/main_classes/quantization#offload-between-cpu-and-gpu for more details. 

In [3]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 42,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

NameError: name 'model' is not defined

# Load datasets

In [9]:
import pandas as pd
from datasets import Dataset
df = pd.read_csv("../data/webnlg_train.csv", header=0, delimiter='\t')

In [10]:
texts = []
for index, row in df.iterrows():
    output = row['Triples']
    p_input = row["NL"]
    triples = output.split('<T>')
    triples = ["<T>" + triple for triple in filter(None, triples)]

    if len(triples) > 1:
        # 1 context
        if len(triples) >= 2:
            context = "".join(triples[0])
            sliced_output = "".join(triples[1:])
            texts.append([context, p_input, sliced_output])

        # 2 context
        if len(triples) >= 3:
            context = "".join(triples[0:2])
            sliced_output = "".join(triples[2:])
            texts.append([context, p_input, sliced_output])

        # 3 context
        if len(triples) >= 4:
            context = "".join(triples[0:3])
            sliced_output = "".join(triples[3:])
            texts.append([context, p_input, sliced_output])

    # Original
    texts.append(["None", p_input, output])

# df
df = pd.DataFrame(texts, columns = ["context", "input", "output"])
df = df.sample(frac=1, random_state=42).reset_index(drop=True)
df

,context,input,output
0,<T>United_States<R>ethnicGroup<S>Native_Americ...,The United States includes the Native American...,"<T>Auburn,_Alabama<R>country<S>United_States<T..."
1,<T>Agnes_Kant<R>nationality<S>Netherlands,The leader of the Netherlands is Mark Rutte wh...,<T>Netherlands<R>leader<S>Mark_Rutte<T>Agnes_K...
2,<T>Alan_Martin_(footballer)<R>club<S>Hamilton_...,The footballer Alan Martin has played for Hami...,<T>Alan_Martin_(footballer)<R>club<S>Aldershot...
3,None,Abilene Regional Airport serves the city of Ab...,<T>Abilene_Regional_Airport<R>cityServed<S>Abi...
4,<T>14th_New_Jersey_Volunteer_Infantry_Monument...,The 14th New Jersey Volunteer Infantry Monumen...,<T>14th_New_Jersey_Volunteer_Infantry_Monument...
...,...,...,...
97131,None,"The birth place of Allan Shivers is Lufkin, Te...","<T>Allan_Shivers<R>birthPlace<S>Lufkin,_Texas"
97132,<T>103_Hera<R>discoverer<S>James_Craig_Watson<...,"James Craig Watson, who discovered 103 Hera, o...",<T>James_Craig_Watson<R>deathCause<S>Peritonitis
97133,<T>Andrew_Rayel<R>associatedBand/associatedMus...,Trance musician Andrew Rayel is associated wit...,<T>Andrew_Rayel<R>associatedBand/associatedMus...
97134,None,Ahmet Ertegun's genre is rhythm and blues.,<T>Ahmet_Ertegun<R>genre<S>Rhythm_and_blues


In [11]:
dataset = Dataset.from_pandas(df)
dataset

Dataset({
    features: ['context', 'input', 'output'],
    num_rows: 97136
})

In [12]:
from trl import DataCollatorForCompletionOnlyLM
prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.
### Instruction:
Extract the most confident information in the sentence below as much as possible, and express the relationships in RDF Triples that complement the existing RDF triples. Do not use information from common sense.
### Existing RDF triples:
{}
### Input:
{}
### Response:
{}"""
EOS_TOKEN = tokenizer.eos_token # must add!

# Format dataset
def format_prompts(examples):
    context = examples["context"]
    inputs       = examples["input"]
    outputs      = examples["output"]
    texts = []
    for instruction, input, output in zip(context, inputs, outputs):
        # Must add EOS_TOKEN, otherwise your generation will go on forever!
        text = prompt.format(instruction, input, output) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }
pass

# response_template = "### Response:"
# collator = DataCollatorForCompletionOnlyLM(response_template, tokenizer=tokenizer)

In [13]:
dataset = dataset.map(format_prompts, batched=True)
dataset

Map:   0%|          | 0/97136 [00:00<?, ? examples/s]

Dataset({
    features: ['context', 'input', 'output', 'text'],
    num_rows: 97136
})

In [14]:
from trl import SFTTrainer, DataCollatorForCompletionOnlyLM
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    dataset_text_field="text",
    # data_collator=collator, # Response only gen
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        # num_train_epochs = 1, # Set this for 1 full training run.
        max_steps = 120,
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 42,
        output_dir = "outputs",
    ),
)

Map (num_proc=2):   0%|          | 0/97136 [00:00<?, ? examples/s]

max_steps is given, it will override any value given in num_train_epochs


In [15]:
#@title Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = NVIDIA GeForce RTX 2060. Max memory = 6.0 GB.
2.57 GB of memory reserved.


In [16]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 97,136 | Num Epochs = 1
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 500
 "-____-"     Number of trainable parameters = 20,766,720


Step,Training Loss
1,2.325000
2,2.404500
3,2.413700
4,2.274600
5,2.022300
6,1.804000
7,1.458300
8,1.219000
9,1.003500
10,0.879600


In [17]:
#@title Show final memory and time stats
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory         /max_memory*100, 3)
lora_percentage = round(used_memory_for_lora/max_memory*100, 3)
print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
print(f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training.")
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

1901.828 seconds used for training.
31.7 minutes used for training.
Peak reserved memory = 4.287 GB.
Peak reserved memory for training = 1.717 GB.
Peak reserved memory % of max memory = 71.45 %.
Peak reserved memory for training % of max memory = 28.617 %.


In [18]:
# # alpaca_prompt = Copied from above
# FastLanguageModel.for_inference(model) # Enable native 2x faster inference
# inputs = tokenizer(
# [
#     prompt.format(
#         "", # input
#         "", # output - leave this blank for generation!
#     )
# ], return_tensors = "pt").to("cuda")
# 
# outputs = model.generate(**inputs, max_new_tokens = 64, use_cache = True)
# tokenizer.batch_decode(outputs)

In [19]:
# alpaca_prompt = Copied from above
FastLanguageModel.for_inference(model) # Enable native 2x faster inference
inputs = tokenizer(
[
    prompt.format(
        "<T>Turn_Me_On_(album)<R>runtime<S>35.1",
        """Turn Me On is a 35.1 minute long album produced by Wharton Tiers that was followed by the album entitled Take it Off.""", # input
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 500)

<bos>Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.
### Instruction:
Extract the most confident information in the sentence below as much as possible, and express the relationships in RDF Triples that complement the existing RDF triples. Do not use information from common sense.
### Existing RDF triples:
<T>Turn_Me_On_(album)<R>runtime<S>35.1
### Input:
Turn Me On is a 35.1 minute long album produced by Wharton Tiers that was followed by the album entitled Take it Off.
### Response:
<T>Turn_Me_On_(album)<R>followedBy<S>Take_It_Off_(album)<T>Turn_Me_On_(album)<R>producer<S>Wharton_Tier<eos>


## Saving

In [20]:
model.save_pretrained("gemma2_2b_context") # Local saving
tokenizer.save_pretrained("gemma2_2b_context")

('gemma2_2b_context/tokenizer_config.json',
 'gemma2_2b_context/special_tokens_map.json',
 'gemma2_2b_context/tokenizer.model',
 'gemma2_2b_context/added_tokens.json',
 'gemma2_2b_context/tokenizer.json')

## Inference (disabled)

if True:
    from unsloth import FastLanguageModel
    model, tokenizer = FastLanguageModel.from_pretrained(
        model_name = "gemma2_2b_test", # YOUR MODEL YOU USED FOR TRAINING
        max_seq_length = 2048,
        dtype = None,
        load_in_4bit = True,
    )
    FastLanguageModel.for_inference(model) # Enable native 2x faster inference

# alpaca_prompt = You MUST copy from above!

prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.
### Instruction:
Using only the information provided in the sentence, extract the relationship between subjects in the sentence below, and express the relationships in RDF Triples.
### Input:
{}
### Response:
{}"""

inputs = tokenizer(
[
    prompt.format(
        "The AP CSA course is a college-level course developed by Collegeboard, it teaches high-school students the basic of programming in Java.", # input
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 100)